In [ ]:
# Install all core and evaluation dependencies for smooth execution
!pip install datasets evaluate transformers huggingface_hub \
  rouge_score bert_score nltk


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00


In [ ]:
# Import core libraries for LLMs, dataset handling, and evaluation
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from huggingface_hub import login
import pandas as pd
from datasets import load_dataset
import evaluate
import time


In [ ]:
# Authenticate with your Hugging Face token (this is Benard's)
print("🔐 Authenticating with Hugging Face...")
login()
print("✅ Authentication successful.\n")


🔐 Authenticating with Hugging Face...
✅ Authentication successful.



In [ ]:
# Define how the chatbot should generate text (length, randomness, etc.)
generation_config = {
    "max_length": 200,
    "temperature": 0.7,
    "top_p": 0.9,
    "do_sample": True
}
print("⚙️ Hyperparameters defined:\n", generation_config, "\n")


⚙️ Hyperparameters defined:
 {'max_length': 200, 'temperature': 0.7, 'top_p': 0.9, 'do_sample': True} 



In [ ]:
# Load the Mistral-7B model and tokenizer from Hugging Face
print("✅ Ready to load the model...")

def load_mistral():
    print("📦 Loading Mistral-7B model and tokenizer...")
    model_name = "mistralai/Mistral-7B-v0.1"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto"
    )
    print("✅ Model and tokenizer loaded successfully.\n")
    return model, tokenizer


✅ Ready to load the model...


In [ ]:
# Create a function to generate chatbot responses based on user input
print("✅ Response generation function loaded.")

def chatbot_response(model, tokenizer, user_input):
    prompt = (
        "You are a helpful and empathetic mental health assistant."
        " Provide thoughtful and supportive responses to the user's concerns.\n\n"
        f"User: {user_input}\n"
        "Assistant:"
    )
    inputs = tokenizer(prompt, return_tensors="pt")
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    output = model.generate(
        **inputs,
        max_length=generation_config["max_length"],
        temperature=generation_config["temperature"],
        top_p=generation_config["top_p"],
        do_sample=generation_config["do_sample"]
    )
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    response = response.split("Assistant:")[-1].strip()
    return response


✅ Response generation function loaded.


In [ ]:
# Load and clean the MentalChat16K dataset (limited to 100 samples)
print("✅ Preprocessing function ready.")

def preprocess_mentalchat16k():
    print("📥 Loading MentalChat16K dataset...")
    dataset = load_dataset("ShenLab/MentalChat16K", split="train[:100]")
    df = pd.DataFrame(dataset)
    df = df.drop_duplicates().dropna()
    df = df.rename(columns={"input": "question", "output": "answer"})
    print("✅ Dataset loaded and preprocessed.\n")
    return df[['question', 'answer']]


✅ Preprocessing function ready.


In [ ]:
# Evaluate chatbot accuracy using ROUGE, BLEU, and BERTScore
print("✅ Evaluation function defined.")

def evaluate_chatbot(model, tokenizer, dataset):
    print("📊 Starting evaluation on 10 samples...")
    references = dataset['answer'].tolist()
    inputs = dataset['question'].tolist()
    predictions = [chatbot_response(model, tokenizer, q) for q in inputs[:10]]

    print("🔍 Loading evaluation metrics...")
    rouge = evaluate.load("rouge")
    bleu = evaluate.load("bleu")
    bertscore = evaluate.load("bertscore")

    print("📈 Calculating scores...")
    rouge_scores = rouge.compute(predictions=predictions, references=references[:10], rouge_types=["rouge1", "rouge2", "rougeL"])
    bleu_score = bleu.compute(predictions=predictions, references=references[:10])
    bert_score = bertscore.compute(predictions=predictions, references=references[:10], lang="en")

    print("\n✅ Evaluation Results:")
    print("ROUGE-1:", rouge_scores['rouge1'])
    print("ROUGE-2:", rouge_scores['rouge2'])
    print("ROUGE-L:", rouge_scores['rougeL'])
    print("BLEU:", bleu_score['bleu'])
    print("BERTScore (F1 avg):", round(sum(bert_score['f1']) / len(bert_score['f1']), 4))
    print("✅ Evaluation completed.\n")


✅ Evaluation function defined.


In [ ]:
# Interface for chatting with the bot directly from Colab
print("✅ Chat interface function is ready.")

def chatbot_interface():
    print("💬 Chatbot ready. Type your mental health questions below.")
    print("Type 'exit' to end the chat.\n")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Goodbye! Stay strong!")
            break
        response = chatbot_response(model, tokenizer, user_input)
        print("Chatbot:", response)


✅ Chat interface function is ready.


In [ ]:
# 🚀 Step 10: Run the Full Chatbot Pipeline
# This step loads the model, prepares the dataset, runs evaluations, and starts the chatbot.
print("🎯 Final step: Executing full pipeline...")

# Run each function directly (no `__main__` needed in Colab)
print("🚀 Initializing system...")
model, tokenizer = load_mistral()
time.sleep(1)

dataset = preprocess_mentalchat16k()
time.sleep(1)

evaluate_chatbot(model, tokenizer, dataset)

chatbot_interface()


🎯 Final step: Executing full pipeline...
🚀 Initializing system...
📦 Loading Mistral-7B model and tokenizer...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



✅ Evaluation Results:
ROUGE-1: 0.2561268571857828
ROUGE-2: 0.08801662092312354
ROUGE-L: 0.1742220500186411
BLEU: 0.0075048182132016385
BERTScore (F1 avg): 0.8665
✅ Evaluation completed.

💬 Chatbot ready. Type your mental health questions below.
Type 'exit' to end the chat.

